In [1]:
import requests
import json
from config import api_key
import pandas as pd
import time

# created a list of the keywords to iterate over
keywords = ['Terrorism',
            'Extremist',
            'Radical',
            'Lone Wolf',
            'Violent Political Parties',
            'Hijacking']

# created a list of begin dates to iterate over the beginning of each year
begin_dates = [20070101, 20080101, 20090101, 20100101, 20110101,
               20120101, 20130101, 20140101, 20150101, 20160101, 20170101]

# created a list of begin dates to iterate over the end of each year
end_dates = [20071231, 20081231, 20091231, 20101231, 20111231,
             20121231, 20131231, 20141231, 20151231, 20161231, 20171231]

In [2]:
# created an empty list to hold the responses
results = []

# jeff note: within each response, there could be a 'number of articles' parameter

# added source = revelance, to populate pg 1 with most 'revelant' articles to the keyword
for kw in keywords:    # 'kw' is keyword

    for begin_date, end_date in zip(begin_dates, end_dates):

        # iterating through the first 5 pages of 'revelant' articles for each year, one keyword at a time
        for page in range(1, 3):

            try:
                url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date={}&end_date={}&q={}&sort=relevance&page={}&api-key={}'.format(begin_date, end_date,
                                                                                                                                                         kw, page, api_key)
                response = requests.get(url).json()

                # sleep requirement of 6 seconds between requests to avoid per min rate limit
                time.sleep(6)

                # printing each keyword and page it iterates through for confirmation
                print(f'{begin_date}-{end_date};  quering page #{page} for {kw}...')

                # where the list of 'article' dicts begins
                for article in response['response']['docs']:
                    # created column declaring the kw used to find that article
                    article['kw'] = kw

                    # created column declaring the page # used to find that article
                    article['page_nbr'] = page

                    # jeff note: create column to declare the "hit_level" used to find that article
                    # article['hits_level'] = # get the hits here

                    # adds the article data created to the existing article data,
                    # appends it to the empty 'results' list
                    results.append(article)
            except:
                print(f'Page {page} for {kw} not found.')

20070101-20071231, Quering Terrorism, page #1
20070101-20071231, Quering Terrorism, page #2
20080101-20081231, Quering Terrorism, page #1
20080101-20081231, Quering Terrorism, page #2
20090101-20091231, Quering Terrorism, page #1
20090101-20091231, Quering Terrorism, page #2
20100101-20101231, Quering Terrorism, page #1
20100101-20101231, Quering Terrorism, page #2
20110101-20111231, Quering Terrorism, page #1
20110101-20111231, Quering Terrorism, page #2
20120101-20121231, Quering Terrorism, page #1
20120101-20121231, Quering Terrorism, page #2
20130101-20131231, Quering Terrorism, page #1
20130101-20131231, Quering Terrorism, page #2
20140101-20141231, Quering Terrorism, page #1
20140101-20141231, Quering Terrorism, page #2
20150101-20151231, Quering Terrorism, page #1
20150101-20151231, Quering Terrorism, page #2
20160101-20161231, Quering Terrorism, page #1
20160101-20161231, Quering Terrorism, page #2
20170101-20171231, Quering Terrorism, page #1
20170101-20171231, Quering Terrori

In [1]:
# created a df from the 'records' list of dicts with article data
df = pd.DataFrame.from_records(results)

df.head()
# what is the difference in "from_records" and "for_dict"??

NameError: name 'pd' is not defined

In [190]:
# created an empty list to hold the new dates
formatted_date = []
formatted_year = []
formatted_month = []

# iterate over each published date in the df 
for date in df['pub_date']:
    
    # converted all pub dates from ints to strings
    date_string = str(date)
    
    # split the date to only contain YYYY-MM-DD
    new_date = date_string[:10]
    
    # split the year and month out from the date
    new_year = date_string[:4]
    new_month = date_string[5:7]
    
    # added new date/month/year to 'formatted date/year/month' list
    formatted_date.append(new_date)
    formatted_month.append(int(new_month))
    formatted_year.append(int(new_year))
    
# added the new date/month/year to the df
df['formatted_date'] = formatted_date
df['formatted_month'] = formatted_month
df['formatted_year'] = formatted_year


In [191]:
# used .map() and used lambda as the function to iterate through each
# row to capture the 'main' title

# created new column 'article title'...
df['article_title'] = df['headline'].map(lambda x: x['main'])

# ...made it equal to the 'main' title for each row


In [192]:
new_df = pd.DataFrame({
    "Keyword Used": df['kw'],
    "Page": df['page_nbr'],
    "Year": df['formatted_year'],
    "Month": df['formatted_month'],
    "Title": df['article_title'],
    "Date": df['formatted_date'],
    "URL": df['web_url'],
    
})

new_df.head()

,Keyword Used,Page,Year,Month,Title,Date,URL
0,Terrorism,1,2007,8,Study Cites Danger of Homegrown Terrorism,2007-08-15,https://www.nytimes.com/2007/08/15/nyregion/15...
1,Terrorism,1,2007,1,Southeast Asians Consider Pact Against Terrorism,2007-01-12,https://www.nytimes.com/2007/01/12/world/asia/...
2,Terrorism,1,2007,11,British Intelligence Chief Sharpens Terrorism ...,2007-11-06,https://www.nytimes.com/2007/11/06/world/europ...
3,Terrorism,1,2007,7,England: Terrorism Suspect Is Sentenced,2007-07-18,https://query.nytimes.com/gst/fullpage.html?re...
4,Terrorism,1,2007,7,British Leader Seeks New Terrorism Laws,2007-07-26,https://www.nytimes.com/2007/07/26/world/europ...


In [194]:
bins = [0, 4, 8, 13]
group_labels = ['Jan-Apr', 'Apr-Aug', 'Sept-Dec']
new_df["Month Group"] = pd.cut(
    new_df['Month'], bins, labels=group_labels).head()